In [ ]:
!pip install face-recognition
import face_recognition
import numpy as np
import os

In [ ]:
def read_pairs_file ( file_path ) :
    pairs = [ ]

    with open(file_path , "r") as file :
        lines = file.readlines()

        for i,line in enumerate(lines) :
            if i==0:
                continue
            parts = line.strip().split("\t")

            if len(parts) == 3 :
                # Case 1: It's a same person pair
                same_person = True
                name1 = parts[ 0 ]
                num1 = int(parts[ 1 ])
                name2 = parts[ 0 ]
                num2 = int(parts[ 2 ])

                file_path = os.path.join(jpn_addr , name1)
                name1 = os.listdir(file_path)[num1-1]
                name1 = os.path.join(file_path , name1)

                file_path = os.path.join(jpn_addr , name2)
                name2 = os.listdir(file_path)[num2-1]
                name2 = os.path.join(file_path , name2)

            elif len(parts) == 4 :
                # Case 2: It's a different person pair
                same_person = False
                name1 = parts[ 0 ]
                num1 = int(parts[ 1 ])
                name2 = parts[ 2 ]
                num2 = int(parts[ 3 ])

                file_path = os.path.join(jpn_addr , name1)
                name1 = os.listdir(file_path)[num1-1]
                name1 = os.path.join(file_path , name1)

                file_path = os.path.join(jpn_addr , name2)
                name2 = os.listdir(file_path)[num2-1]
                name2 = os.path.join(file_path , name2)

            else :
                raise ValueError("Invalid format in pairs.txt")

            pairs.append((same_person , name1 , num1 , name2 , num2))

    return pairs

def y_or_n(p1_addr,q2_addr,kk=0.6):
    """
    输入两张图片和阈值，返回是否匹配
    :param p1_addr: 图片1的地址
    :param q2_addr: 图片2的地址
    :param kk: 阈值
    :return: 是否同一个人
    """
    p_obama_image = face_recognition.load_image_file(p1_addr)
    q_obama_image = face_recognition.load_image_file(q2_addr)
    p_face_encoding = face_recognition.face_encodings(p_obama_image)
    q_face_encoding = face_recognition.face_encodings(q_obama_image)
    if(len(p_face_encoding)==0 or len(q_face_encoding)==0):
        return "NOT" , True
    else:
        p_face_encoding = p_face_encoding[0]
        q_face_encoding = q_face_encoding[0]
    known_encodings = [p_face_encoding]
    face_distances = face_recognition.face_distance(known_encodings , q_face_encoding)
    if(face_distances<=kk):
        return str(face_distances),True
    else:
        return str(face_distances),False

In [ ]:
pairs_file = r"/kaggle/input/pairs-txt-02/pairs.txt"  # 请替换为实际文件路径
jpn_addr = r'/kaggle/input/lfw-yuantu-2'

# 使用示例
pairs_data = read_pairs_file(pairs_file)
long = len(pairs_data)
kk_l = [0.46,0.48,0.50,0.52,0.54,0.56,0.58,0.60,0.62,0.64,0.66,0.68,0.70]
for kk in np.arange(0.60,0.66,0.005):
    yes = 0
    num = 0
    T = 0
    for i,pair in enumerate(pairs_data) :
        pairs_val,name1,name2= pair[ 0 ],pair[ 1 ],pair[ 3 ]
        loss,output = y_or_n(name1,name2,kk)
        if loss == "NOT":
            #print(f"{i} | 没有人脸 | 正确率为：{1.0*yes/(num)}")
            T= T+1
            continue
        # 正常识别的数量
        num = num +1
        if pairs_val== output:
            yes = yes + 1
#             print(f"{i} | 预测正确 | 正确率为：{1.0*yes/(num)}")
#         else :
#             print(f"{i} | 预测错误 | 正确率为：{1.0*yes/(num)}")
    print(f"阈值为:{kk} | 总共输入了{long}张图片对 | 正常识别{num}张 | 正确识别{yes}张| 正确率为:{1.0*yes/num} | 跳过了{T}张 | 跳过率为{1.0*T/long}")